In [1]:
import numpy as np
from scipy.linalg import expm

def build_generator_Q(k: int = 12) -> np.ndarray:
    """
    Build a generator matrix Q for the augmented CTMC that tracks:
      - phase in {A1, A2, B1, B2}
      - count n in {0,1,...,k-1} representing N(t) = n
      - one absorbing state representing N(t) >= k

    Here k=12 corresponds to computing P(N(t) < 12).
    Time unit: months.
    """
    # Phase order: 0=A1, 1=A2, 2=B1, 3=B2

    # Subgenerator for the PH interarrival time (transient-to-transient transitions)
    S = np.array([
        [-1.0,   1.0,  0.0,  0.0],   # A1 -> A2 at rate 1
        [ 0.0,  -1.0,  0.9,  0.1],   # A2 -> B1 at 0.9, A2 -> B2 at 0.1 (total rate 1)
        [ 0.0,   0.0, -1.0,  0.0],   # B1 completes at rate 1 (handled as jump to next count)
        [ 0.0,   0.0,  0.0, -1.0/11.0]  # B2 completes at rate 1/11
    ])

    # Completion (absorption) rates from each phase
    s = np.array([0.0, 0.0, 1.0, 1.0/11.0])  # only B1 and B2 can complete

    n_levels = k                 # counts 0..k-1 are explicit
    dim = 4 * n_levels + 1       # +1 absorbing state for count >= k
    absorbing = dim - 1

    Q = np.zeros((dim, dim))

    def idx(phase: int, n: int) -> int:
        return 4 * n + phase

    # Fill Q
    for n in range(n_levels):
        # Within-level phase evolution
        Q[4*n:4*n+4, 4*n:4*n+4] = S

        # Completion transitions increment the count and reset to phase A1
        if n < n_levels - 1:
            # from (B1,n) -> (A1,n+1) at rate 1
            # from (B2,n) -> (A1,n+1) at rate 1/11
            for phase in range(4):
                if s[phase] != 0.0:
                    Q[idx(phase, n), idx(0, n+1)] += s[phase]
        else:
            # if already at n=k-1 (i.e., 11 when k=12), next completion goes to absorbing ">=k"
            for phase in range(4):
                if s[phase] != 0.0:
                    Q[idx(phase, n), absorbing] += s[phase]

    # Absorbing state has no exits (row of zeros), which is correct.
    print(Q[:10,:10])
    return Q

def prob_N_less_than_k(t_months: float = 60.0, k: int = 12) -> float:
    """
    Compute P(N(t) < k) using matrix exponential of the generator Q.
    Start condition: right after a completion -> (A1, n=0).
    """
    Q = build_generator_Q(k=k)
    dim = Q.shape[0]
    absorbing = dim - 1

    # Initial distribution: in phase A1 with count 0
    p0 = np.zeros(dim)
    p0[0] = 1.0

    # Evolve distribution: p(t) = p(0) exp(Q t)
    pt = p0 @ expm(Q * t_months)
    # Probability that we have NOT reached count >= k by time t
    return 1.0 - pt[absorbing]

if __name__ == "__main__":
    t_months = 60.0  # 5 years
    k = 12           # want P(N(t) < 12)
    p = prob_N_less_than_k(t_months=t_months, k=k)
    print(f"P(N(5 years) < 12) = {p:.6f}")


[[-1.          1.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.         -1.          0.9         0.1         0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.         -1.          0.          1.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         -0.09090909  0.09090909  0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.         -1.          1.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -1.
   0.9         0.1         0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
  -1.          0.          1.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.         -0.09090909  0.09090909  0.        ]
 [ 0.          0.          0.          0.          0.          0

In [9]:

def prob_N_less_than_k_new(t_months: float = 60.0, k: int = 12) -> float:
    """
    Compute P(N(t) < k) using matrix exponential of the generator Q.
    Start condition: right after a completion -> (A1, n=0).
    """
    Q = build_generator_Q(k=k)
    dim = Q.shape[0]
    absorbing = dim - 1

    # Initial distribution: in phase A1 with count 0
    p0 = np.zeros(dim-1)
    p0[0] = 1.0
    S = Q[:-1,:-1]
    ones = np.ones((S.shape[0],1))
    # Evolve distribution: p(t) = p(0) exp(Q t)
    pt = p0 @ expm(S * t_months) @ ones
    # Probability that we have NOT reached count >= k by time t
    return 1.0 - pt


if __name__ == "__main__":
    t_months = 60.0  # 5 years
    k = 12           # want P(N(t) < 12)
    p = prob_N_less_than_k_new(t_months=t_months, k=k)
    print(f"P(N(5 years) < 12) = {p:.6f}")


[[-1.          1.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.         -1.          0.9         0.1         0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.         -1.          0.          1.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         -0.09090909  0.09090909  0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.         -1.          1.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -1.
   0.9         0.1         0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
  -1.          0.          1.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.         -0.09090909  0.09090909  0.        ]
 [ 0.          0.          0.          0.          0.          0

TypeError: unsupported format string passed to numpy.ndarray.__format__

#Q18

In [ ]:
from sympy import *
l = latex(Matrix([[-1,1,0,0,0,0,0,0],[0,-1,0.9,0.1,0,0,0,0],[0.65,0,-1,0,0.35,0,0,0],[S(0.65)/11,0,0,-S(1)/11,S(0.35)/11,0,0,0],[0,0,0,0,-1,1,0,0],[0,0,0,0,0,-1,0.9,0.1],[1,0,0,0,0,0,-1,0],[S(1)/11,0,0,0,0,0,0,-S(1)/11]]))
print(l)

\left[\begin{matrix}-1 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & -1 & 0.9 & 0.1 & 0 & 0 & 0 & 0\\0.65 & 0 & -1 & 0 & 0.35 & 0 & 0 & 0\\0.0590909090909091 & 0 & 0 & - \frac{1}{11} & 0.0318181818181818 & 0 & 0 & 0\\0 & 0 & 0 & 0 & -1 & 1 & 0 & 0\\0 & 0 & 0 & 0 & 0 & -1 & 0.9 & 0.1\\1 & 0 & 0 & 0 & 0 & 0 & -1 & 0\\\frac{1}{11} & 0 & 0 & 0 & 0 & 0 & 0 & - \frac{1}{11}\end{matrix}\right]


#Question 20
## Here we remove the idle to busy jumps to model time busy -> idle -> busy

In [28]:
T = (Matrix([[-1,1,0,0,0,0,0,0],[0,-1,0.9,0.1,0,0,0,0],[0.65,0,-1,0,0,0,0,0],[S(0.65)/11,0,0,-S(1)/11,0,0,0,0],[0,0,0,0,-1,1,0,0],[0,0,0,0,0,-1,0.9,0.1],[1,0,0,0,0,0,-1,0],[S(1)/11,0,0,0,0,0,0,-S(1)/11]]))
alpha_busy = Matrix([0,0,0,0,1,0,0,0]).T
ones = Matrix([[1],[1],[1],[1],[1],[1],[1],[1]])
U = (-T).inv()
m1 = alpha_busy @ U @ ones #PH-type paper Section 1.4.3 corollary 1 #first order moment
m2 = 2*alpha_busy @ U @ U @ ones #second order moment
var = m2 - m1**2
print("mean",m1), print("var",var)

mean Matrix([[15.4285714285714]])
var Matrix([[177.469387755102]])


(None, None)

In [24]:
ones.shape,alpha_busy.shape,T.shape


((8, 1), (8, 1), (8, 8))